In [6]:
# import the random module
import random
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
from datetime import datetime
import requests
import time
import pandas as pd
from scipy.stats import linregress # this is for linear regression

In [7]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [8]:
# add the lat and longs to a list
coordinates = list(lat_lngs)

In [12]:
# create a list for holding the cities
cities = []

# identify the nearest city for each lat and long combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

#print the city count to confirm sufficient count
len(cities)

764

In [13]:
# Starting URL for Weather Map API Call.
# import the API key
from config import weather_api_key
from datetime import datetime
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [14]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [15]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          'Current Description': description})

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [16]:
# Tally the number of cities in the city_data array of dictionaries
len(city_data)

704

In [18]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Current Description']
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,Rikitea,PF,-23.1203,-134.9692,75.76,66,66,broken clouds
1,Tuatapere,NZ,-46.1333,167.6833,52.83,94,100,overcast clouds
2,Quatre Cocos,MU,-20.2078,57.7625,78.10,72,36,scattered clouds
3,Ahipara,NZ,-35.1667,173.1667,71.06,67,88,overcast clouds
4,Port Hardy,CA,50.6996,-127.4199,42.80,81,75,light intensity shower rain
5,Mirnyy,RU,62.5353,113.9611,-15.88,96,100,overcast clouds
6,Chiang Khong,TH,20.2613,100.4046,55.72,90,0,clear sky
7,Mandalgovi,MN,45.7625,106.2708,20.50,43,43,scattered clouds
8,Busselton,AU,-33.6500,115.3333,77.04,47,1,clear sky
9,Goderich,CA,43.7501,-81.7165,30.90,80,100,overcast clouds


In [19]:
# Create the output file (CSV).
output_data_file = "WeatherPy_database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")